In [1]:
#we may need some code in the ../python directory and/or matplotlib styles
import sys
import os
sys.path.append('../python/')

#set up matplotlib
os.environ['MPLCONFIGDIR'] = '../mplstyles'
print(os.environ['MPLCONFIGDIR'])
import matplotlib as mpl
from matplotlib import pyplot as plt
#got smarter about the mpl config: see mplstyles/ directory
plt.style.use('standard')
print(mpl.__version__) 
print(mpl.get_configdir())


#fonts
# Set the font dictionaries (for plot title and axis titles)
title_font = {'fontname':'Arial', 'size':'16', 'color':'black', 'weight':'normal',
              'verticalalignment':'bottom'} # Bottom vertical alignment for more space
axis_font = {'fontname':'Arial', 'size':'32'}
legend_font = {'fontname':'Arial', 'size':'22'}

#fonts global settings
mpl.rc('font',family=legend_font['fontname'])


#set up numpy
import numpy as np

../mplstyles
3.0.3
/home/phys/villaa/analysis/misc/nrFano_paper2019/mplstyles


In [33]:
import prob_dist as pd
import EdwRes as er
import time
import imp
imp.reload(pd)

#get the resolutions                                                                              
sigHv,sigIv,sigQerv,sigH_NRv,sigI_NRv,sigQnrv = \
 er.getEdw_det_res('GGA3',4.0,'data/edw_res_data.txt',aH=0.0381,C=None) 

fF = pd.QEr_v2_2D_fast(sigHv,sigIv,4.0,3.0,0.00000001,Qbar=lambda x: 0.16*x**0.18)                            
gF = pd.expband_2D(fF,(1/18.0),1.5)


start = time.time()
fF(0.1,10.0,10.0)
end = time.time()
print('Analytic Expression: {:1.5f} sec.'.format(end-start))

start = time.time()
gF(0.1,10.0)
end = time.time()
print('Numerical Integration: {:1.5f} sec.'.format(end-start))

start = time.time()
pd.sigmomEdw(10,band='NR',label='GGA3',F=0.000001,V=4.0,aH=0.0381,alpha=(1/18.0))
end = time.time()
print('Normalization and Integration: {:1.5f} sec.'.format(end-start))

start = time.time()
np.sin(np.pi)
end = time.time()
print('sin function: {:1.5f} sec.'.format(end-start))

Analytic Expression: 0.00104 sec.
Numerical Integration: 0.18725 sec.
Normalization and Integration: 20.42548 sec.
sin function: 0.00009 sec.


In [30]:
#check how the means and norms change
from scipy.integrate import quad

for Er in np.arange(10,200,50):
  norm = quad(gF,-1,4,args=(Er,),epsabs=1e-10)[0]
  Qdist = lambda Q: (1/norm)*gF(Q,Er)
  meanfun = lambda Q: Q*Qdist(Q)
  mean = quad(meanfun,-1,4,epsabs=1e-10)[0]
  print('energy: {:2.6f}'.format(Er))
  print('calc mean: {:2.6f}; Qmean: {:2.6f}'.format(mean,0.16*Er**0.18))
  print('calc norm: {:2.8f};'.format(norm))
  #sigfun = lambda Q: Q**2*Qdist(Q)
  #q2 = quad(sigfun,-1,4)[0]

energy: 10.000000
calc mean: 0.242241; Qmean: 0.242170
calc norm: 10.32813952;
energy: 60.000000
calc mean: 0.334199; Qmean: 0.334339
calc norm: 0.64214199;
energy: 110.000000
calc mean: 0.372412; Qmean: 0.372881
calc norm: 0.04000521;
energy: 160.000000
calc mean: 0.398058; Qmean: 0.398898
calc norm: 0.00249732;


In [32]:
alpha = (1/18.0)
print('norm ratio: {:2.3f}; expected ratio: {:2.3f}'.format(0.64214199/10.32813952,np.exp(-alpha*60)/np.exp(-alpha*10)))
print('norm ratio: {:2.3f}; expected ratio: {:2.3f}'.format(0.04000521/10.32813952,np.exp(-alpha*110)/np.exp(-alpha*10)))
print('norm ratio: {:2.8f}; expected ratio: {:2.8f}'.format(0.00249732/10.32813952,np.exp(-alpha*160)/np.exp(-alpha*10)))

norm ratio: 0.062; expected ratio: 0.062
norm ratio: 0.004; expected ratio: 0.004
norm ratio: 0.00024180; expected ratio: 0.00024037
